In [72]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt


In [73]:
IMAGE_SIZE =[224, 224]

In [74]:
train_path = 'chest_xray/train'
valid_path = 'chest_xray/test'

In [75]:
#Instatiate the model
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)

In [76]:
#Skip training of the existing weights
for layer in vgg.layers:
    layer.trainable = False

In [77]:
#final classes
folders = glob('chest_xray/train')

In [78]:
#adding final layer
x= Flatten()(vgg.output)
prediction = Dense(len(folders), activation ='softmax')(x) #adding last layer with output len(folder ==2), activavtion func softmax and appending the flat layer output, x

In [79]:
#creating a model
model = Model(inputs = vgg.input, output = prediction)

#model structure
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

C:\Users\Kweronda\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [80]:
#model optimization methods
model.compile(loss= 'categorical_crossentropy', optimizer = 'adam', metrics =['accuracy'])

In [81]:
#image preprocessing
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255, shear_range =0.2, zoom_range =0.2, horizontal_flip=True)
test_datagen =ImageDataGenerator(rescale =1./255)

training_set = train_datagen.flow_from_directory('chest_xray/train', 
                                                target_size = (224, 224),
                                                batch_size =32,
                                                 class_mode = 'categorical'
                                                )

test_set = test_datagen.flow_from_directory('chest_xray/test',
                                           target_size = (224, 224),
                                            batch_size =32,
                                            class_mode = 'categorical'
                                           )

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [84]:
#fitting the model
r = model.fit_generator(
    training_set, 
                        validation_data = test_set, 
                        epochs =5,
    steps_per_epoch =len(training_set),
    validation_steps = len(test_set)
)

Epoch 1/5


ValueError: Error when checking target: expected dense_5 to have shape (1,) but got array with shape (2,)

In [ ]:
#plotting loss
plt.plot(r.hostory['loss'], label = 'train loss')
plt.plot(r.history['val_loss'], label = 'val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

#plot acurracies
plt.plot(r.history['acc'], label='train acc')
plt.plot(r.history['val_acc'], label = 'val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
#saving model
import tensorflow as tf
from keras.models import load_model
model.save('xray_vgg16.h5')